##### Importing the relevant libraries to run the code

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

#Ensures output is step by step.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

##### Importing the relevant dataset from uci machine learning database

In [2]:
data = pd.read_excel('Online Retail.xlsx')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
print(data)

       InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
5         536365     22752         SET 7 BABUSHKA NESTING BOXES         2   
6         536365     21730    GLASS STAR FROSTED T-LIGHT HOLDER         6   
7         536366     22633               HAND WARMER UNION JACK         6   
8         536366     22632            HAND WARMER RED POLKA DOT         6   
9         536367     84879        ASSORTED COLOUR BIRD ORNAMENT        32   
10        536367     22745           POPPY'S PLAYHOUSE BEDROOM          6   
11        536367     22748            POPPY'S PLAYHOUSE KITCHEN         6   

##### Cleaning up the data by removing the irrelevant columns or correcting the format of the values in the columns.

In [4]:
#1.We remove the spaces in the values under the column Description
#We use the function .string.strip()
data['Description'] = data['Description'].str.strip()

In [5]:
#2. We remove the rows where the Invoice Number column is empty.
data.dropna(axis=0 , subset=['InvoiceNo'], inplace= True)

In [6]:
#3. We then change the datatype for the column Invoice Number to string
data['InvoiceNo'] = data['InvoiceNo'].astype('str')

In [7]:
#4. We then remove the credit transaction that have 'C' in there invoice numbers
data = data[~data['InvoiceNo'].str.contains('C')]
print(data)

       InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
5         536365     22752         SET 7 BABUSHKA NESTING BOXES         2   
6         536365     21730    GLASS STAR FROSTED T-LIGHT HOLDER         6   
7         536366     22633               HAND WARMER UNION JACK         6   
8         536366     22632            HAND WARMER RED POLKA DOT         6   
9         536367     84879        ASSORTED COLOUR BIRD ORNAMENT        32   
10        536367     22745            POPPY'S PLAYHOUSE BEDROOM         6   
11        536367     22748            POPPY'S PLAYHOUSE KITCHEN         6   

##### We then perform a one-hot encoding on all transactions.
##### One-Hot encoding is used to deal with the issue where the model is confused thinking that a column has some sort of hierarchy due to label-encoding which is not the case.
##### One-Hot Encoding in this case will be used to classify items in a specific Invoice Number.
##### Where the item appears a 1 is put and a 0 otherwise.

In [8]:
#For purposes of reducing the dataset the dataset will be selected 
#On sales of France and consequently compared with those made in Germany.
#From the code after the sum of the column Quantity we unstack it.
#By unstacking we mean reshaping the dataset as seen in the output.
#We then reset the index. 
#Here another level or column is added to the dataset instead of arranging the columns in the manner they were in before. 

FSales = (data[data['Country'] =="France"]
                .groupby(['InvoiceNo', 'Description'])['Quantity']
                .sum().unstack().reset_index().fillna(0)
                .set_index('InvoiceNo'))

FSales.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
#Due to the very many zeros and even for easy interpretation of the
#dataset we change everything that is greater than 1 to 1 and anything less as a 0.
#We use a function to do the replacement. 

def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

FSales_sets = FSales.applymap(encode_units)
FSales_sets.drop('POSTAGE', inplace=True, axis=1)

##### We next generate item sets that have a certain support and in this case  we use 7% support. 

In [10]:
#Using the apriori library from Mlxtend.
#In this case it takes in the dataset, the support percentage needed and sets using the original column names as true as parameters.
frequent_itemsets = apriori(FSales_sets, min_support=0.07, use_colnames=True)

##### We then generate the rules depending on what item sets passed  the support, confidence and lift.

In [11]:
#We use the association rule library from Mlxtend
#In this case it takes in the frequent items gotten above,the metric as lift and a minimum threshold of 1 as parameters.
#From the output the antecedants would be what imply something
#The consequents would be what is implied.
#E.g X-> Y(X implies Y).
#The X is the antecedant and Y will be the consequent.

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102041,0.096939,0.073980,0.725000,7.478947,0.064088,3.283859
1,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.096939,0.102041,0.073980,0.763158,7.478947,0.064088,3.791383
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
4,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.102041,0.094388,0.073980,0.725000,7.681081,0.064348,3.293135


##### Knowing the confidence level and lift is not enough. We need to understand what a combination of confidence or lift means.

In [12]:
#So in this case we will filter the combinations that have lift of 6 or greater
#And a confidence level of 0.8 or 80% or greater.

rules[ (rules['lift'] >= 6) & (rules['confidence'] >= 0.8) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
17,(SET/6 RED SPOTTY PAPER PLATES),(SET/20 RED RETROSPOT PAPER NAPKINS),0.127551,0.132653,0.102041,0.800000,6.030769,0.085121,4.336735
18,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.137755,0.127551,0.122449,0.888889,6.968889,0.104878,7.852041
19,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127551,0.137755,0.122449,0.960000,6.968889,0.104878,21.556122
20,"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.127551,0.099490,0.975000,7.644000,0.086474,34.897959
21,"(SET/6 RED SPOTTY PAPER CUPS, SET/6 RED SPOTTY...",(SET/20 RED RETROSPOT PAPER NAPKINS),0.122449,0.132653,0.099490,0.812500,6.125000,0.083247,4.625850
22,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.137755,0.099490,0.975000,7.077778,0.085433,34.489796


In [13]:
#From the above Output, Green Alarms seem to be purchased together with Red ones
#Through getting the sums of each as per the dataset the sales of one could be increased through recommendations.

FSales['ALARM CLOCK BAKELIKE GREEN'].sum()
FSales['ALARM CLOCK BAKELIKE RED'].sum()

340.0

316.0

##### Of importance would also be to see the sales of a different country and compare.

In [14]:
#For this case Germany is now going to be used.
#From the code after the sum of the column Quantity we unstack it.
#By unstacking we mean reshaping the dataset as seen in the output.
#We then reset the index. 
#Here another level or column is added to the dataset instead of arranging the columns in the manner they were in before. 

GSales = (data[data['Country'] =="Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))
GSales.head()


Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,...,YULETIDE IMAGES GIFT WRAP SET,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
#Using the replacement code we used for France we call it to do the same replacement on the German Dataset
GSales_sets = GSales.applymap(encode_units)
GSales_sets.drop('POSTAGE', inplace=True, axis=1)

In [16]:
#We then get the frequent items sets in Germany.
frequent_itemsetsG = apriori(GSales_sets, min_support=0.05, use_colnames=True)

In [17]:
# We similarily create rules and their associations
rulesG = association_rules(frequent_itemsetsG, metric="lift", min_threshold=1)

In [18]:
#So we will filter the combinations that have lift of 4 or greater
#And a confidence level of 0.5 or 50% or greater.

rulesG[ (rulesG['lift'] >= 4) & (rulesG['confidence'] >= 0.5)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.115974,0.137856,0.067834,0.584906,4.242887,0.051846,2.076984
7,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN WOODLAND ANIMALS),0.107221,0.137856,0.061269,0.571429,4.145125,0.046488,2.011670
10,(RED RETROSPOT CHARLOTTE BAG),(WOODLAND CHARLOTTE BAG),0.070022,0.126915,0.059081,0.843750,6.648168,0.050194,5.587746


##### As compared to France, the two items that seem to be bought together the most are Red Retrosport Charlotte Bag and  Woodland Charlotte Bag. 